# Exercício 1 - Enunciado


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
    Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.
    ```
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', r', t − q × t' 
    OUTPUT r, s, t
    ````
    1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
    2. Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
        Prove que o programa nunca atinge o estado de erro
    3. Prove que a relação de Bézout $\,$ $\,a*s + b*t = r\,$ é um invariante do algoritmo.

# Exercício 1 - Solução

a)

In [26]:
from pysmt.shortcuts import *
from pysmt.typing import *

N = 32
mx = BV(2**N-1, width=N)
zero = BV(0, width=N)
um = BV(1, width=N)

a = Symbol("a", BVType(N))
b = Symbol("b", BVType(N))
r = Symbol("r", BVType(N))
r_prime = Symbol("r_prime", BVType(N))
s = Symbol("s", BVType(N))
s_prime = Symbol("s_prime", BVType(N))
t = Symbol("t", BVType(N))
t_prime = Symbol("t_prime", BVType(N))
q = Symbol("q", BVType(N))

init = And(
    BVUGT(a, zero),
    BVUGT(b, zero),
    Equals(r, a),
    Equals(r_prime, b),
    Equals(s, um),
    Equals(s_prime, zero),
    Equals(t, zero),
    Equals(t_prime, um),
    # Implies(Equals(r_prime,zero), And(Equals(r, a), Equals(s,um), Equals(t,zero))),
)

variables = [a, b, r, r_prime, s, s_prime, t, t_prime, q]

while_condition = Not(Equals(r_prime,zero))
body = And(
    Equals(q, BVUDiv(r, r_prime)),
    Equals(r, BVMul(r, BVMul(q, r_prime))),
    Equals(s, BVSub(s, BVMul(q, s_prime))),
    Equals(t, BVSub(t, BVMul(q, t_prime))),
    Equals(r, r_prime),
    Equals(s, s_prime),
    Equals(t, t_prime)
)

trans = Implies(while_condition, body)

b)

In [27]:
def hasOverflowed(var):
    return BVUGT(var, mx)

erro = Or(
    Equals(r, zero),
    hasOverflowed(a),
    hasOverflowed(b),
    hasOverflowed(r),
    hasOverflowed(r_prime),
    hasOverflowed(s),
    hasOverflowed(s_prime),
    hasOverflowed(t),
    hasOverflowed(t_prime),
    hasOverflowed(q)
)

if is_unsat(And(init, trans, erro)):
    print("Nunca chega a estado de erro")
else:
    print("Chega a estado de erro")

Nunca chega a estado de erro


c)

In [28]:
bezout = Equals(BVAdd(BVMul(a, s_prime), BVMul(b, t_prime)), r_prime)
trans = Implies(while_condition, And(body, Not(bezout)))

if is_unsat(And(init, trans)):
    print("A relacao de bezout e uma invariante")
else:
    print("A relacao de bezout nao e uma invariante")

A relacao de bezout e uma invariante
